In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAll([('spark.executor.memory', '4g'),
                           ('spark.executor.cores', '4'),
                           ('spark.cores.max', '4'),
                           ('spark.driver.memory','10g')])
conf.set("spark.app.name", "Barinov lab3") 
spark = SparkSession.builder.config(conf=conf).appName("Barinov lab3").getOrCreate()

In [3]:
spark

In [4]:
from pyspark.sql.types import FloatType, DoubleType, StringType, ArrayType, IntegerType
from pyspark.sql.functions import *
import pyspark.sql.functions as f
spark.conf.set("spark.sql.crossJoin.enabled", True)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 1)

In [5]:
!hdfs dfs -ls /labs/slaba03/

Found 4 items
-rw-r--r--   3 hdfs hdfs   91066524 2021-02-27 22:12 /labs/slaba03/laba03_items.csv
-rw-r--r--   3 hdfs hdfs   29965581 2021-02-27 22:12 /labs/slaba03/laba03_test.csv
-rw-r--r--   3 hdfs hdfs   74949368 2021-02-27 22:12 /labs/slaba03/laba03_train.csv
-rw-r--r--   3 hdfs hdfs  871302535 2021-02-27 22:12 /labs/slaba03/laba03_views_programmes.csv


In [6]:
items = spark.read.option("header","true").csv("/labs/slaba03/laba03_items.csv", sep='\t')
items.show(2, vertical=True)

-RECORD 0-------------------------------------------
 item_id                     | 65667                
 channel_id                  | null                 
 datetime_availability_start | 1970-01-01T00:00:00Z 
 datetime_availability_stop  | 2018-01-01T00:00:00Z 
 datetime_show_start         | null                 
 datetime_show_stop          | null                 
 content_type                | 1                    
 title                       | на пробах только ... 
 year                        | 2013.0               
 genres                      | Эротика              
 region_id                   | null                 
-RECORD 1-------------------------------------------
 item_id                     | 65669                
 channel_id                  | null                 
 datetime_availability_start | 1970-01-01T00:00:00Z 
 datetime_availability_stop  | 2018-01-01T00:00:00Z 
 datetime_show_start         | null                 
 datetime_show_stop          | null           

In [7]:
items.printSchema()

root
 |-- item_id: string (nullable = true)
 |-- channel_id: string (nullable = true)
 |-- datetime_availability_start: string (nullable = true)
 |-- datetime_availability_stop: string (nullable = true)
 |-- datetime_show_start: string (nullable = true)
 |-- datetime_show_stop: string (nullable = true)
 |-- content_type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- region_id: string (nullable = true)



In [8]:
items = items.withColumn("item_id", items["item_id"].cast(IntegerType()))
items = items.withColumn("channel_id", items["channel_id"].cast(IntegerType()))
items = items.withColumn("year", items["year"].cast(IntegerType()))
items = items.withColumn("region_id", items["region_id"].cast(IntegerType()))

In [9]:
test = spark.read.option("header","true").csv("/labs/slaba03/laba03_test.csv", sep=',')
test.show(5)

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  94814|    null|
|   1654|  93629|    null|
|   1654|   9980|    null|
|   1654|  95099|    null|
|   1654|  11265|    null|
+-------+-------+--------+
only showing top 5 rows



In [10]:
test.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- purchase: string (nullable = true)



In [11]:
test = test.withColumn("user_id", test["user_id"].cast(IntegerType()))
test = test.withColumn("item_id", test["item_id"].cast(IntegerType()))
test = test.withColumn("purchase", test["purchase"].cast(FloatType()))

In [12]:
train = spark.read.option("header","true").csv("/labs/slaba03/laba03_train.csv", sep=',')
train.show(5)

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  74107|       0|
|   1654|  89249|       0|
|   1654|  99982|       0|
|   1654|  89901|       0|
|   1654| 100504|       0|
+-------+-------+--------+
only showing top 5 rows



In [13]:
train.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- purchase: string (nullable = true)



In [14]:
train = train.withColumn("user_id", train["user_id"].cast(StringType()))
train = train.withColumn("item_id", train["item_id"].cast(StringType()))
train = train.withColumn("purchase", train["purchase"].cast(FloatType()))

In [16]:
views = spark.read.option("header","true").csv("/labs/slaba03/laba03_views_programmes.csv", sep=',')
views.show(5)

+-------+-------+----------+----------+---------+
|user_id|item_id|  ts_start|    ts_end|item_type|
+-------+-------+----------+----------+---------+
|      0|7101053|1491409931|1491411600|     live|
|      0|7101054|1491412481|1491451571|     live|
|      0|7101054|1491411640|1491412481|     live|
|      0|6184414|1486191290|1486191640|     live|
|    257|4436877|1490628499|1490630256|     live|
+-------+-------+----------+----------+---------+
only showing top 5 rows



In [17]:
views.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- ts_start: string (nullable = true)
 |-- ts_end: string (nullable = true)
 |-- item_type: string (nullable = true)



In [18]:
views = views.withColumn("user_id", views["user_id"].cast(IntegerType()))
views = views.withColumn("item_id", views["item_id"].cast(IntegerType()))
views = views.withColumn("ts_start", views["ts_start"].cast(IntegerType()))
views = views.withColumn("ts_end", views["ts_end"].cast(IntegerType()))
views.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- ts_start: integer (nullable = true)
 |-- ts_end: integer (nullable = true)
 |-- item_type: string (nullable = true)



# EDA

In [718]:
train.select('user_id','item_id').distinct().count()

5032624

In [492]:
test.select('user_id').distinct().count()

1941

**Множества юзеров и айтемов из трейн и тест совпадают:**

In [493]:
train.join(test, test.user_id == train.user_id, how='inner').select(train.user_id).distinct().count()

1941

In [494]:
train.select('item_id').distinct().count()

3704

In [495]:
test.select('item_id').distinct().count()

3704

**Множества айтемов из трейн и тест совпадают:**

In [496]:
train.join(test, test.item_id == train.item_id, how='inner').select(train.item_id).distinct().count()

3704

**Но нет совпадений юзер+айтем, ну и прекрасно:**

In [497]:
train.join(test, (test.item_id == train.item_id) & (test.user_id == train.user_id),\
           how='inner').select(train.item_id).distinct().count()

0

**Есть ли в test передачи, по которым доступна мета-инфа в items? ЕСТЬ **

In [498]:
test.join(items, test.item_id == items.item_id,\
           how='inner').select(test.item_id).distinct().count()

3704

**Есть ли в train передачи, по которым доступна мета-инфа в items? ЕСТЬ**

In [499]:
train.join(items, train.item_id == items.item_id,\
           how='inner').select(train.item_id).distinct().count()

3704

In [500]:
items.select(items.channel_id).distinct().count()

208

In [501]:
views.groupBy('item_type').count().show()

+---------+--------+
|item_type|   count|
+---------+--------+
|     live|17704201|
|      pvr| 3141406|
+---------+--------+



**В Test уже учтен только платный контент content_type**

In [719]:
test.join(items, test.item_id == items.item_id,\
           how='left').groupBy('content_type').count().show()

+------------+-------+
|content_type|  count|
+------------+-------+
|           1|2156840|
+------------+-------+



**В Train тоже только платный**

In [720]:
train.join(items, train.item_id == items.item_id,\
           how='left').groupBy('content_type').count().show()

+------------+-------+
|content_type|  count|
+------------+-------+
|           1|5032624|
+------------+-------+



**хз что такое, но пусть будет**

In [503]:
train.join(items, train.item_id == items.item_id,\
           how='inner').groupBy('purchase','datetime_availability_start').count().show()

+--------+---------------------------+-------+
|purchase|datetime_availability_start|  count|
+--------+---------------------------+-------+
|     1.0|       2017-01-01T00:00:00Z|      1|
|     1.0|       1970-01-01T00:00:00Z|  10903|
|     0.0|       1970-01-01T00:00:00Z|5018987|
|     0.0|       2017-01-01T00:00:00Z|   2733|
+--------+---------------------------+-------+



In [504]:
train.join(items, train.item_id == items.item_id,\
           how='inner').groupBy('purchase','datetime_availability_stop').count().show()

+--------+--------------------------+-------+
|purchase|datetime_availability_stop|  count|
+--------+--------------------------+-------+
|     0.0|      2018-01-01T00:00:00Z| 126702|
|     0.0|      2017-08-21T00:00:00Z|   1398|
|     0.0|      2099-12-31T21:00:00Z|4880210|
|     0.0|      2018-12-31T00:00:00Z|  13410|
|     1.0|      2018-12-31T00:00:00Z|    159|
|     1.0|      2099-12-31T21:00:00Z|   9991|
|     1.0|      2017-08-21T00:00:00Z|      5|
|     1.0|      2018-01-01T00:00:00Z|    749|
+--------+--------------------------+-------+



**datetime_show_start и stop бесполезны**

In [505]:
train.join(items, train.item_id == items.item_id,\
           how='inner').groupBy('purchase','datetime_show_start').count().show()

+--------+-------------------+-------+
|purchase|datetime_show_start|  count|
+--------+-------------------+-------+
|     1.0|               null|  10904|
|     0.0|               null|5021720|
+--------+-------------------+-------+



In [526]:
train.join(items, train.item_id == items.item_id,\
           how='inner').groupBy('purchase','datetime_show_stop').count().show()

+--------+------------------+-------+
|purchase|datetime_show_stop|  count|
+--------+------------------+-------+
|     1.0|              null|  10904|
|     0.0|              null|5021720|
+--------+------------------+-------+



**region_id тоже бесполезен**

In [519]:
train.join(items, train.item_id == items.item_id,\
           how='inner').groupBy('purchase','region_id').count().show()

+--------+---------+-------+
|purchase|region_id|  count|
+--------+---------+-------+
|     1.0|     null|  10904|
|     0.0|     null|5021720|
+--------+---------+-------+



**получается item_type тоже бесполезен**

In [521]:
train.join(views, (train.item_id == views.item_id) & (train.user_id == views.user_id),\
           how='left').groupBy('purchase','item_type').count().show()

+--------+---------+-------+
|purchase|item_type|  count|
+--------+---------+-------+
|     1.0|     null|  10904|
|     0.0|     null|5021720|
+--------+---------+-------+



**получается channel_id тоже бесполезен**

In [527]:
train.join(items, train.item_id == items.item_id,\
           how='left').groupBy('purchase','channel_id').count().show()

+--------+----------+-------+
|purchase|channel_id|  count|
+--------+----------+-------+
|     1.0|      null|  10904|
|     0.0|      null|5021720|
+--------+----------+-------+



# Давайте слепим датасет

**Количество просмотров live**

In [19]:
cnt_live = views.filter(f.col('item_type') == 'live').groupBy('user_id', 'item_type').count()

cnt_live = cnt_live.withColumnRenamed("count","cnt_live")
cnt_live.show(3)

+-------+---------+--------+
|user_id|item_type|cnt_live|
+-------+---------+--------+
| 785542|     live|       2|
| 785679|     live|     557|
| 785685|     live|     330|
+-------+---------+--------+
only showing top 3 rows



**Количество просмотров cvr**

In [20]:
cnt_pvr = views.filter(f.col('item_type') == 'pvr').groupBy('user_id', 'item_type').count()

cnt_pvr = cnt_pvr.withColumnRenamed("count","cnt_pvr")
cnt_pvr.show(3)

+-------+---------+-------+
|user_id|item_type|cnt_pvr|
+-------+---------+-------+
| 786175|      pvr|     11|
| 786990|      pvr|    116|
| 787349|      pvr|     44|
+-------+---------+-------+
only showing top 3 rows



**Ср.время просмотра юзера**

In [21]:
watch_time_user = views.withColumn("user", views.ts_end - views.ts_start)\
    .groupBy('user_id')\
    .mean()\
    .select("user_id", "avg(user)")
watch_time_user = watch_time_user.withColumnRenamed("avg(user)","time_user")

watch_time_user.show(3)

+-------+-----------------+
|user_id|        time_user|
+-------+-----------------+
| 870787|2953.016441005803|
| 870827|6508.959183673469|
| 870928|4118.955555555555|
+-------+-----------------+
only showing top 3 rows



**Ср.время просмотра айтема**

In [22]:
watch_time_item = views.withColumn("item", views.ts_end - views.ts_start)\
    .groupBy('item_id')\
    .mean()\
    .select("item_id", "avg(item)")
watch_time_item = watch_time_item.withColumnRenamed("avg(item)","time_item")

watch_time_item.show(3)

+-------+------------------+
|item_id|         time_item|
+-------+------------------+
|6526786|3098.6971153846152|
|7076695| 3088.930473372781|
|6604291|           1777.92|
+-------+------------------+
only showing top 3 rows



**Количество и доля покупок у юзера**

In [23]:
sum_buys_user = train.groupBy("user_id").sum("purchase").withColumnRenamed("sum(purchase)","sum_buys_user")
sum_buys_user.show(3)

+-------+-------------+
|user_id|sum_buys_user|
+-------+-------------+
| 867363|          1.0|
| 882935|          2.0|
| 889974|          0.0|
+-------+-------------+
only showing top 3 rows



In [24]:
part_buys_user = train.groupBy("user_id").agg(sum("purchase") / count("purchase"))\
    .withColumnRenamed("(sum(purchase) / count(purchase))","part_buys_user") 
part_buys_user.show(3)

+-------+--------------------+
|user_id|      part_buys_user|
+-------+--------------------+
| 921852|3.789314134141720...|
| 927169|0.003464203233256351|
| 929499|0.002714230321830167|
+-------+--------------------+
only showing top 3 rows



In [25]:
sum_buys_item = train.groupBy("item_id").sum("purchase")\
.withColumnRenamed("sum(purchase)","sum_buys_item")
sum_buys_item.show(2)

+-------+-------------+
|item_id|sum_buys_item|
+-------+-------------+
|  98725|          1.0|
|    691|          0.0|
+-------+-------------+
only showing top 2 rows



In [26]:
part_buys_item = train.groupBy("item_id").agg(sum("purchase") / count("purchase"))\
    .withColumnRenamed("(sum(purchase) / count(purchase))","part_buys_item") 
part_buys_item.show(3)

+-------+--------------------+
|item_id|      part_buys_item|
+-------+--------------------+
|  98725|7.369196757553427E-4|
|    691|                 0.0|
|   2136| 7.22543352601156E-4|
+-------+--------------------+
only showing top 3 rows



**Доли онлайн просмотров для юзеров и айтемов**

In [27]:
live_user = views.withColumn("is_live", when(f.col("item_type") == "live", 1)\
    .otherwise(0))\
    .groupBy("user_id")\
    .agg(sum("is_live") / count("is_live"))\
    .withColumnRenamed("(sum(is_live) / count(is_live))", "live_user")
live_user.show(3)

+-------+------------------+
|user_id|         live_user|
+-------+------------------+
| 785637|               1.0|
| 786134|0.9691119691119691|
| 786310|0.5486725663716814|
+-------+------------------+
only showing top 3 rows



In [28]:
live_item = views.withColumn("is_live", when(f.col("item_type") == "live", 1)\
    .otherwise(0))\
    .groupBy("item_id")\
    .agg(sum("is_live") / count("is_live"))\
    .withColumnRenamed("(sum(is_live) / count(is_live))", "live_item")
live_item.show(3)

+-------+------------------+
|item_id|         live_item|
+-------+------------------+
|6470621|0.9384615384615385|
|7101164|0.9852941176470589|
|6867283|               1.0|
+-------+------------------+
only showing top 3 rows



**Смотрибельность передач и юзеров по времени**

In [29]:
watch_time_user = views.withColumn("user", views.ts_end - views.ts_start)\
    .groupBy('user_id')\
    .sum()\
    .select("user_id", "sum(user)")
watch_time_user = watch_time_user.withColumnRenamed("sum(user)","watch_time_user")

watch_time_user.show(3)

+-------+---------------+
|user_id|watch_time_user|
+-------+---------------+
| 561425|          37699|
| 612390|           4406|
| 612597|         687126|
+-------+---------------+
only showing top 3 rows



In [30]:
avg_time_user = views.withColumn("user", views.ts_end - views.ts_start)\
    .groupBy('user_id')\
    .mean()\
    .select("user_id", "avg(user)")
avg_time_user = avg_time_user.withColumnRenamed("avg(user)","avg_time_user")

avg_time_user.show(3)

+-------+-----------------+
|user_id|    avg_time_user|
+-------+-----------------+
| 870787|2953.016441005803|
| 870827|6508.959183673469|
| 870928|4118.955555555555|
+-------+-----------------+
only showing top 3 rows



In [31]:
watch_time_item = views.withColumn("item", views.ts_end - views.ts_start)\
    .groupBy('item_id')\
    .sum()\
    .select("item_id", "sum(item)")
watch_time_item = watch_time_item.withColumnRenamed("sum(item)","watch_time_item")

watch_time_item.show(3)

+-------+---------------+
|item_id|watch_time_item|
+-------+---------------+
|6526786|        1933587|
|7076695|        2088117|
|6604291|          88896|
+-------+---------------+
only showing top 3 rows



In [32]:
avg_time_item = views.withColumn("item", views.ts_end - views.ts_start)\
    .groupBy('item_id')\
    .mean()\
    .select("item_id", "avg(item)")
avg_time_item = avg_time_item.withColumnRenamed("avg(item)","avg_time_item")

avg_time_item.show(3)

+-------+------------------+
|item_id|     avg_time_item|
+-------+------------------+
|6470621|1640.7307692307693|
|7101164| 6031.573529411765|
|6867283|            1376.6|
+-------+------------------+
only showing top 3 rows



**Смотрибельность передач и юзеров по количеству**

In [33]:
cnt_user = views.groupBy('user_id').count()

cnt_user = cnt_user.withColumnRenamed("count","cnt_user")
cnt_user.show(3)

+-------+--------+
|user_id|cnt_user|
+-------+--------+
| 785637|     259|
| 786134|     259|
| 786310|     113|
+-------+--------+
only showing top 3 rows



In [34]:
cnt_item = views.groupBy('item_id').count()

cnt_item = cnt_item.withColumnRenamed("count","cnt_item")
cnt_item.show(3)

+-------+--------+
|item_id|cnt_item|
+-------+--------+
|6922406|    1558|
|6550191|      72|
|6559523|     828|
+-------+--------+
only showing top 3 rows



**Собираем датасет**

In [35]:
%%time
train_df = train.join(items, ['item_id'], how='left')\
    .join(views, ['user_id', 'item_id'], how='left')\
    .join(cnt_user, ['user_id'], how='left')\
    .join(cnt_item, ['item_id'], how='left')\
    .join(watch_time_user, ['user_id'], how='left')\
    .join(watch_time_item, ['item_id'], how='left')\
    .join(avg_time_user, ['user_id'], how='left')\
    .join(avg_time_item, ['item_id'], how='left')\
    .join(sum_buys_user, ['user_id'], how='left')\
    .join(part_buys_user, ['user_id'], how='left')\
    .join(sum_buys_item, ['item_id'], how='left')\
    .join(part_buys_item, ['item_id'], how='left')\
    .join(live_user, ['user_id'], how='left')\
    .join(live_item, ['item_id'], how='left')\
    .select(
        train.user_id,
        train.item_id,
        items.year,
        cnt_user.cnt_user,
        cnt_item.cnt_item,
        watch_time_user.watch_time_user,
        watch_time_item.watch_time_item,
        avg_time_user.avg_time_user,
        avg_time_item.avg_time_item,
        sum_buys_user.sum_buys_user,
        part_buys_user.part_buys_user,
        sum_buys_item.sum_buys_item,
        part_buys_item.part_buys_item,
        live_item.live_item,
        live_user.live_user,
        split(items.genres,",")[0].alias("genre1"),
        split(items.genres,",")[1].alias("genre2"),
        split(items.genres,",")[2].alias("genre3"),
        split(items.genres,",")[3].alias("genre4"),
        split(items.genres,",")[4].alias("genre5"),
        year(items.datetime_availability_start).alias("year_start"),
        year(items.datetime_availability_stop).alias("year_stop"),
        train.purchase
).cache()

train_df.show(1, truncate=False, vertical=True)

-RECORD 0-------------------------------
 user_id         | 754230               
 item_id         | 8389                 
 year            | 1981                 
 cnt_user        | 1164                 
 cnt_item        | null                 
 watch_time_user | 2256455              
 watch_time_item | null                 
 avg_time_user   | 1938.5352233676977   
 avg_time_item   | null                 
 sum_buys_user   | 72.0                 
 part_buys_user  | 0.027575641516660282 
 sum_buys_item   | 8.0                  
 part_buys_item  | 0.005979073243647235 
 live_item       | null                 
 live_user       | 0.39776632302405496  
 genre1          | Мультфильмы          
 genre2          | Детские              
 genre3          | Союзмультфильм       
 genre4          | Наши                 
 genre5          | null                 
 year_start      | 1970                 
 year_stop       | 2099                 
 purchase        | 0.0                  
only showing top

In [36]:
train_df.count()

5032624

In [37]:
%%time
test_df = test.join(items, ['item_id'], how='left')\
    .join(views, ['user_id', 'item_id'], how='left')\
    .join(cnt_user, ['user_id'], how='left')\
    .join(cnt_item, ['item_id'], how='left')\
    .join(watch_time_user, ['user_id'], how='left')\
    .join(watch_time_item, ['item_id'], how='left')\
    .join(avg_time_user, ['user_id'], how='left')\
    .join(avg_time_item, ['item_id'], how='left')\
    .join(sum_buys_user, ['user_id'], how='left')\
    .join(part_buys_user, ['user_id'], how='left')\
    .join(sum_buys_item, ['item_id'], how='left')\
    .join(part_buys_item, ['item_id'], how='left')\
    .join(live_user, ['user_id'], how='left')\
    .join(live_item, ['item_id'], how='left')\
    .select(
        test.user_id,
        test.item_id,
        items.year,
        cnt_user.cnt_user,
        cnt_item.cnt_item,
        watch_time_user.watch_time_user,
        watch_time_item.watch_time_item,
        avg_time_user.avg_time_user,
        avg_time_item.avg_time_item,
        sum_buys_user.sum_buys_user,
        part_buys_user.part_buys_user,
        sum_buys_item.sum_buys_item,
        part_buys_item.part_buys_item,
        live_item.live_item,
        live_user.live_user,
        split(items.genres,",")[0].alias("genre1"),
        split(items.genres,",")[1].alias("genre2"),
        split(items.genres,",")[2].alias("genre3"),
        split(items.genres,",")[3].alias("genre4"),
        split(items.genres,",")[4].alias("genre5"),
        year(items.datetime_availability_start).alias("year_start"),
        year(items.datetime_availability_stop).alias("year_stop")
).cache()

test_df.show(1, truncate=False, vertical=True)

-RECORD 0--------------------------------
 user_id         | 822709                
 item_id         | 8389                  
 year            | 1981                  
 cnt_user        | 870                   
 cnt_item        | null                  
 watch_time_user | 2856524               
 watch_time_item | null                  
 avg_time_user   | 3283.36091954023      
 avg_time_item   | null                  
 sum_buys_user   | 1.0                   
 part_buys_user  | 3.7893141341417203E-4 
 sum_buys_item   | 8.0                   
 part_buys_item  | 0.005979073243647235  
 live_item       | null                  
 live_user       | 0.5183908045977011    
 genre1          | Мультфильмы           
 genre2          | Детские               
 genre3          | Союзмультфильм        
 genre4          | Наши                  
 genre5          | null                  
 year_start      | 1970                  
 year_stop       | 2099                  
only showing top 1 row

CPU times:

In [38]:
test_df.count()

2156840

In [39]:
train_df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- cnt_user: long (nullable = true)
 |-- cnt_item: long (nullable = true)
 |-- watch_time_user: long (nullable = true)
 |-- watch_time_item: long (nullable = true)
 |-- avg_time_user: double (nullable = true)
 |-- avg_time_item: double (nullable = true)
 |-- sum_buys_user: double (nullable = true)
 |-- part_buys_user: double (nullable = true)
 |-- sum_buys_item: double (nullable = true)
 |-- part_buys_item: double (nullable = true)
 |-- live_item: double (nullable = true)
 |-- live_user: double (nullable = true)
 |-- genre1: string (nullable = true)
 |-- genre2: string (nullable = true)
 |-- genre3: string (nullable = true)
 |-- genre4: string (nullable = true)
 |-- genre5: string (nullable = true)
 |-- year_start: integer (nullable = true)
 |-- year_stop: integer (nullable = true)
 |-- purchase: float (nullable = true)



In [40]:
test_df = test_df.withColumn("user_id", test_df["user_id"].cast(IntegerType()))
test_df = test_df.withColumn("item_id", test_df["item_id"].cast(IntegerType()))
test_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- cnt_user: long (nullable = true)
 |-- cnt_item: long (nullable = true)
 |-- watch_time_user: long (nullable = true)
 |-- watch_time_item: long (nullable = true)
 |-- avg_time_user: double (nullable = true)
 |-- avg_time_item: double (nullable = true)
 |-- sum_buys_user: double (nullable = true)
 |-- part_buys_user: double (nullable = true)
 |-- sum_buys_item: double (nullable = true)
 |-- part_buys_item: double (nullable = true)
 |-- live_item: double (nullable = true)
 |-- live_user: double (nullable = true)
 |-- genre1: string (nullable = true)
 |-- genre2: string (nullable = true)
 |-- genre3: string (nullable = true)
 |-- genre4: string (nullable = true)
 |-- genre5: string (nullable = true)
 |-- year_start: integer (nullable = true)
 |-- year_stop: integer (nullable = true)



In [41]:
train_df = train_df.withColumn("user_id", train_df["user_id"].cast(IntegerType()))
train_df = train_df.withColumn("item_id", train_df["item_id"].cast(IntegerType()))
train_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- cnt_user: long (nullable = true)
 |-- cnt_item: long (nullable = true)
 |-- watch_time_user: long (nullable = true)
 |-- watch_time_item: long (nullable = true)
 |-- avg_time_user: double (nullable = true)
 |-- avg_time_item: double (nullable = true)
 |-- sum_buys_user: double (nullable = true)
 |-- part_buys_user: double (nullable = true)
 |-- sum_buys_item: double (nullable = true)
 |-- part_buys_item: double (nullable = true)
 |-- live_item: double (nullable = true)
 |-- live_user: double (nullable = true)
 |-- genre1: string (nullable = true)
 |-- genre2: string (nullable = true)
 |-- genre3: string (nullable = true)
 |-- genre4: string (nullable = true)
 |-- genre5: string (nullable = true)
 |-- year_start: integer (nullable = true)
 |-- year_stop: integer (nullable = true)
 |-- purchase: float (nullable = true)



In [44]:
from pyspark.ml.feature import StringIndexer
indexer1 = StringIndexer(inputCol="genre1", outputCol="genre1_index")
indexed1 = indexer1.setHandleInvalid("keep").fit(train_df).transform(train_df)

In [45]:
indexer2 = StringIndexer(inputCol="genre2", outputCol="genre2_index")
indexed2 = indexer2.setHandleInvalid("keep").fit(indexed1).transform(indexed1)

indexer3 = StringIndexer(inputCol="genre3", outputCol="genre3_index")
indexed3 = indexer3.setHandleInvalid("keep").fit(indexed2).transform(indexed2)

indexer4 = StringIndexer(inputCol="genre4", outputCol="genre4_index")
indexed4 = indexer4.setHandleInvalid("keep").fit(indexed3).transform(indexed3)

indexer5 = StringIndexer(inputCol="genre5", outputCol="genre5_index")
indexed5 = indexer5.setHandleInvalid("keep").fit(indexed4).transform(indexed4)

In [46]:
indexed5.show(1, vertical=True)

-RECORD 0-------------------------------
 user_id         | 754230               
 item_id         | 8389                 
 year            | 1981                 
 cnt_user        | 1164                 
 cnt_item        | null                 
 watch_time_user | 2256455              
 watch_time_item | null                 
 avg_time_user   | 1938.5352233676977   
 avg_time_item   | null                 
 sum_buys_user   | 72.0                 
 part_buys_user  | 0.027575641516660282 
 sum_buys_item   | 8.0                  
 part_buys_item  | 0.005979073243647235 
 live_item       | null                 
 live_user       | 0.39776632302405496  
 genre1          | Мультфильмы          
 genre2          | Детские              
 genre3          | Союзмультфильм       
 genre4          | Наши                 
 genre5          | null                 
 year_start      | 1970                 
 year_stop       | 2099                 
 purchase        | 0.0                  
 genre1_index   

In [47]:
train_idf = indexed5.drop('genre1', 'genre2', 'genre3', 'genre4', 'genre5')

In [48]:
train_idf.show(1, vertical=True)

-RECORD 0-------------------------------
 user_id         | 754230               
 item_id         | 8389                 
 year            | 1981                 
 cnt_user        | 1164                 
 cnt_item        | null                 
 watch_time_user | 2256455              
 watch_time_item | null                 
 avg_time_user   | 1938.5352233676977   
 avg_time_item   | null                 
 sum_buys_user   | 72.0                 
 part_buys_user  | 0.027575641516660282 
 sum_buys_item   | 8.0                  
 part_buys_item  | 0.005979073243647235 
 live_item       | null                 
 live_user       | 0.39776632302405496  
 year_start      | 1970                 
 year_stop       | 2099                 
 purchase        | 0.0                  
 genre1_index    | 6.0                  
 genre2_index    | 12.0                 
 genre3_index    | 7.0                  
 genre4_index    | 1.0                  
 genre5_index    | 35.0                 
only showing top

In [49]:
test_indexer1 = StringIndexer(inputCol="genre1", outputCol="genre1_index")
test_test_indexed1 = test_indexer1.setHandleInvalid("keep").fit(test_df).transform(test_df)

test_indexer2 = StringIndexer(inputCol="genre2", outputCol="genre2_index")
test_test_indexed2 = test_indexer2.setHandleInvalid("keep").fit(test_test_indexed1).transform(test_test_indexed1)

test_indexer3 = StringIndexer(inputCol="genre3", outputCol="genre3_index")
test_test_indexed3 = test_indexer3.setHandleInvalid("keep").fit(test_test_indexed2).transform(test_test_indexed2)

test_indexer4 = StringIndexer(inputCol="genre4", outputCol="genre4_index")
test_test_indexed4 = test_indexer4.setHandleInvalid("keep").fit(test_test_indexed3).transform(test_test_indexed3)

test_indexer5 = StringIndexer(inputCol="genre5", outputCol="genre5_index")
test_test_indexed5 = test_indexer5.setHandleInvalid("keep").fit(test_test_indexed4).transform(test_test_indexed4)

In [50]:
test_test_indexed5.show(1, vertical=True)

-RECORD 0-------------------------------
 user_id         | 822709               
 item_id         | 8389                 
 year            | 1981                 
 cnt_user        | 870                  
 cnt_item        | null                 
 watch_time_user | 2856524              
 watch_time_item | null                 
 avg_time_user   | 3283.36091954023     
 avg_time_item   | null                 
 sum_buys_user   | 1.0                  
 part_buys_user  | 3.789314134141720... 
 sum_buys_item   | 8.0                  
 part_buys_item  | 0.005979073243647235 
 live_item       | null                 
 live_user       | 0.5183908045977011   
 genre1          | Мультфильмы          
 genre2          | Детские              
 genre3          | Союзмультфильм       
 genre4          | Наши                 
 genre5          | null                 
 year_start      | 1970                 
 year_stop       | 2099                 
 genre1_index    | 6.0                  
 genre2_index   

In [51]:
test_idf = test_test_indexed5.drop('genre1', 'genre2', 'genre3', 'genre4', 'genre5')

In [52]:
test_idf.show(1, vertical=True)

-RECORD 0-------------------------------
 user_id         | 822709               
 item_id         | 8389                 
 year            | 1981                 
 cnt_user        | 870                  
 cnt_item        | null                 
 watch_time_user | 2856524              
 watch_time_item | null                 
 avg_time_user   | 3283.36091954023     
 avg_time_item   | null                 
 sum_buys_user   | 1.0                  
 part_buys_user  | 3.789314134141720... 
 sum_buys_item   | 8.0                  
 part_buys_item  | 0.005979073243647235 
 live_item       | null                 
 live_user       | 0.5183908045977011   
 year_start      | 1970                 
 year_stop       | 2099                 
 genre1_index    | 6.0                  
 genre2_index    | 12.0                 
 genre3_index    | 7.0                  
 genre4_index    | 1.0                  
 genre5_index    | 35.0                 
only showing top 1 row



In [53]:
%%time
from pyspark.ml.feature import VectorAssembler

feat_cols = train_idf.columns
feat_cols.remove('purchase')
vectorAssembler = VectorAssembler(inputCols = feat_cols, outputCol = 'features')
vdf_train = vectorAssembler.setHandleInvalid('keep').transform(train_idf)
vdf_test = vectorAssembler.setHandleInvalid('keep').transform(test_idf)

CPU times: user 23.7 ms, sys: 7.93 ms, total: 31.7 ms
Wall time: 418 ms


In [54]:
vdf_train.show(1, vertical=True)

-RECORD 0-------------------------------
 user_id         | 754230               
 item_id         | 8389                 
 year            | 1981                 
 cnt_user        | 1164                 
 cnt_item        | null                 
 watch_time_user | 2256455              
 watch_time_item | null                 
 avg_time_user   | 1938.5352233676977   
 avg_time_item   | null                 
 sum_buys_user   | 72.0                 
 part_buys_user  | 0.027575641516660282 
 sum_buys_item   | 8.0                  
 part_buys_item  | 0.005979073243647235 
 live_item       | null                 
 live_user       | 0.39776632302405496  
 year_start      | 1970                 
 year_stop       | 2099                 
 purchase        | 0.0                  
 genre1_index    | 6.0                  
 genre2_index    | 12.0                 
 genre3_index    | 7.0                  
 genre4_index    | 1.0                  
 genre5_index    | 35.0                 
 features       

In [55]:
vdf_test.show(1, vertical=True)

-RECORD 0-------------------------------
 user_id         | 822709               
 item_id         | 8389                 
 year            | 1981                 
 cnt_user        | 870                  
 cnt_item        | null                 
 watch_time_user | 2856524              
 watch_time_item | null                 
 avg_time_user   | 3283.36091954023     
 avg_time_item   | null                 
 sum_buys_user   | 1.0                  
 part_buys_user  | 3.789314134141720... 
 sum_buys_item   | 8.0                  
 part_buys_item  | 0.005979073243647235 
 live_item       | null                 
 live_user       | 0.5183908045977011   
 year_start      | 1970                 
 year_stop       | 2099                 
 genre1_index    | 6.0                  
 genre2_index    | 12.0                 
 genre3_index    | 7.0                  
 genre4_index    | 1.0                  
 genre5_index    | 35.0                 
 features        | [822709.0,8389.0,... 
only showing top

In [56]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(labelCol="purchase",
                   maxBins=70,
                   maxIter=10,
                   maxDepth=10)

In [57]:
%%time
gbt_model = gbt.fit(vdf_train.select('features', 'purchase'))
gbt_predictions = gbt_model.transform(vdf_test)
gbt_predictions.show(5)

+-------+-------+----+--------+--------+---------------+---------------+------------------+-------------+-------------+--------------------+-------------+--------------------+---------+------------------+----------+---------+------------+------------+------------+------------+------------+--------------------+--------------------+
|user_id|item_id|year|cnt_user|cnt_item|watch_time_user|watch_time_item|     avg_time_user|avg_time_item|sum_buys_user|      part_buys_user|sum_buys_item|      part_buys_item|live_item|         live_user|year_start|year_stop|genre1_index|genre2_index|genre3_index|genre4_index|genre5_index|            features|          prediction|
+-------+-------+----+--------+--------+---------------+---------------+------------------+-------------+-------------+--------------------+-------------+--------------------+---------+------------------+----------+---------+------------+------------+------------+------------+------------+--------------------+--------------------+
|

In [60]:
sort_cols = ['user_id', 'item_id']

In [61]:
pred_final = gbt_predictions.select('user_id', 'item_id', 'prediction').orderBy(sort_cols, ascending=True)
pred_final.show()

+-------+-------+--------------------+
|user_id|item_id|          prediction|
+-------+-------+--------------------+
|   1654|    336|2.082252448161448...|
|   1654|    678|3.361754396954419E-5|
|   1654|    691|0.001706885084463...|
|   1654|    696|0.001251808925392...|
|   1654|    763|4.820450841900690...|
|   1654|    795|0.001141267186160...|
|   1654|    861|5.003302186070495E-4|
|   1654|   1137|0.001638953050254...|
|   1654|   1159|0.001203658726586...|
|   1654|   1428|4.875351991191196...|
|   1654|   1685|0.002669678782550067|
|   1654|   1686|0.001115900346186...|
|   1654|   1704|0.003799082847896...|
|   1654|   2093|-9.50410382736278...|
|   1654|   2343|0.001164050544991...|
|   1654|   2451|-4.53053375288120...|
|   1654|   2469|0.005377543023339494|
|   1654|   2603|0.001105730273660...|
|   1654|   2609|5.672998138122466E-6|
|   1654|   2621|0.002669245515051362|
+-------+-------+--------------------+
only showing top 20 rows



In [75]:
pred_final.summary().show()

+-------+-----------------+------------------+--------------------+
|summary|          user_id|           item_id|          prediction|
+-------+-----------------+------------------+--------------------+
|  count|          2156840|           2156840|             2156840|
|   mean|869652.3733920922| 66896.00283609354|0.002368068419827...|
| stddev|60706.51616333811|35227.831307046414| 0.01615443935100325|
|    min|             1654|               326| -0.3239834442297699|
|    25%|           846164|             65668|4.370825463907960...|
|    50%|           885247|             79856|1.383871110444908E-4|
|    75%|           908588|             93606|9.628860939496383E-4|
|    max|           941450|            104165|  1.4431132445932913|
+-------+-----------------+------------------+--------------------+



In [76]:
pred_mod = pred_final\
.withColumn('prediction_mod',when(pred_final.prediction < 0, 0)\
            .otherwise(when(pred_final.prediction > 1, 1)\
                       .otherwise(pred_final.prediction)))
pred_mod.show()

+-------+-------+--------------------+--------------------+
|user_id|item_id|          prediction|      prediction_mod|
+-------+-------+--------------------+--------------------+
|   1654|    336|2.082252448161448...|2.082252448161448...|
|   1654|    678|3.361754396954419E-5|3.361754396954419E-5|
|   1654|    691|0.001706885084463...|0.001706885084463...|
|   1654|    696|0.001251808925392...|0.001251808925392...|
|   1654|    763|4.820450841900690...|4.820450841900690...|
|   1654|    795|0.001141267186160...|0.001141267186160...|
|   1654|    861|5.003302186070495E-4|5.003302186070495E-4|
|   1654|   1137|0.001638953050254...|0.001638953050254...|
|   1654|   1159|0.001203658726586...|0.001203658726586...|
|   1654|   1428|4.875351991191196...|4.875351991191196...|
|   1654|   1685|0.002669678782550067|0.002669678782550067|
|   1654|   1686|0.001115900346186...|0.001115900346186...|
|   1654|   1704|0.003799082847896...|0.003799082847896...|
|   1654|   2093|-9.50410382736278...|  

In [77]:
pred_mod.summary().show()

+-------+-----------------+-----------------+--------------------+--------------------+
|summary|          user_id|          item_id|          prediction|      prediction_mod|
+-------+-----------------+-----------------+--------------------+--------------------+
|  count|          2156840|          2156840|             2156840|             2156840|
|   mean|869652.3733920922|66896.00283609354|0.002368068419827...|0.002407431428232...|
| stddev|60706.51616333833|35227.83130704641| 0.01615443935100325|0.015944526798910386|
|    min|             1654|              326| -0.3239834442297699|                 0.0|
|    25%|           846164|            65668|4.370825463907960...|4.370825463907960...|
|    50%|           885247|            79856|1.383237904492889...|1.383237904492889...|
|    75%|           908588|            93606|9.628860939496383E-4|9.628860939496383E-4|
|    max|           941450|           104165|  1.4431132445932913|                 1.0|
+-------+-----------------+-----

In [78]:
pred_mod = pred_mod.drop('prediction').withColumnRenamed('prediction_mod','purchase')

In [79]:
pred_mod.show()

+-------+-------+--------------------+
|user_id|item_id|            purchase|
+-------+-------+--------------------+
|   1654|    336|2.082252448161448...|
|   1654|    678|3.361754396954419E-5|
|   1654|    691|0.001706885084463...|
|   1654|    696|0.001251808925392...|
|   1654|    763|4.820450841900690...|
|   1654|    795|0.001141267186160...|
|   1654|    861|5.003302186070495E-4|
|   1654|   1137|0.001638953050254...|
|   1654|   1159|0.001203658726586...|
|   1654|   1428|4.875351991191196...|
|   1654|   1685|0.002669678782550067|
|   1654|   1686|0.001115900346186...|
|   1654|   1704|0.003799082847896...|
|   1654|   2093|                 0.0|
|   1654|   2343|0.001164050544991...|
|   1654|   2451|                 0.0|
|   1654|   2469|0.005377543023339494|
|   1654|   2603|0.001105730273660...|
|   1654|   2609|5.672998138122466E-6|
|   1654|   2621|0.002669245515051362|
+-------+-------+--------------------+
only showing top 20 rows



In [80]:
pred_mod.toPandas().to_csv('lab03.csv')

In [325]:
sc.stop()